In [ ]:
# ===============================================================
# 03_kerr_flux_sim.ipynb  — Kerr-flux with Kretschmann coupling
# ===============================================================

# Cell 1 ─ 安裝與載入 -------------------------------------------------
!pip install --quiet numpy scipy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# Cell 2 ─ 物理常數與黑洞參數 -----------------------------------------
G, c  = 6.67430e-11, 2.99792458e8
M_sun = 1.98847e30
hbar  = 1.054571817e-34
M_pl  = np.sqrt(hbar * c / G)

M_bh   = 4.0e6 * M_sun   # Sgr A*
a_star = 0.5

r_g_m  = 2*G*M_bh/c**2
print(f"Schwarzschild radius r_g ≈ {r_g_m/1e3:.2e} km ,  a* = {a_star}")

# Cell 3 ─ 無因次化與耦合 λ̃(x) ---------------------------------------
a_hat = a_star            # a/r_g
l_P   = np.sqrt(hbar*G/c**3)
supp  = (l_P / r_g_m)**2  # Planck 壓制 ~10⁻⁹⁰

Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

κ = 2.0e-2                # 同論文
def lambda_tilde(x):
    # Kretschmann (leading term)  K̂ = 48/x^6
    K_hat = 48.0 / x**6
    return κ * K_hat * supp

# Cell 4 ─ ODE:  ψ, χ = dψ/dx ---------------------------------------
def rhs(x, y):
    ψ, χ = y
    num = x**2 * lambda_tilde(x) * ψ * (ψ**2 - 1)
    den = x**2 * Δ(x)
    dxd = 2*x*Δ(x) + x**2 * dΔ(x)
    return [χ, (num - dxd*χ)/den]

# Cell 5 ─ 射擊法 ----------------------------------------------------
x_outer = 1e3  # r̂ = 1000
def integrate(A):
    ψ0, χ0 = A/x_outer, -A/x_outer**2
    sol = solve_ivp(rhs, (x_outer, 2.0),
                    [ψ0, χ0],
                    t_eval=np.geomspace(x_outer, 2.0, 1200),
                    method="Radau", rtol=1e-9, atol=1e-12)
    if not sol.success:
        return None, None
    return sol.t[::-1], 2*sol.y[0][::-1]   # x, ε

target = 1e-8                  # 論文預期 ∼10⁻⁹
bestA, bestErr = None, np.inf
for A in np.logspace(-10, -2, 80):
    x, ε = integrate(A)
    if x is None:  continue
    err = abs(ε[np.argmin(abs(x-10))] - target)
    if err < bestErr:
        bestA, bestErr = A, err

if bestA is None:
    raise RuntimeError("No solution found; widen A-scan.")
print(f"Selected A = {bestA:.3e},  ε(10 r_g) error = {bestErr:.2e}")

x_arr, ε_arr = integrate(bestA)

# Cell 6 ─ Analytic ansatz -----------------------------------------
r_s0_hat = (8.5e3*3.085677e19)/r_g_m
boost    = 1.0 / (1.0 - a_hat/x_arr)
ε_ansatz = 2*np.tanh(x_arr/(r_s0_hat*boost))

# Cell 7 ─ 圖形 ------------------------------------------------------
plt.figure(figsize=(8,5))
plt.loglog(x_arr, ε_arr, label="Numerical ε(r)")
plt.loglog(x_arr, ε_ansatz, '--', label="Analytic ansatz")
plt.axvline(10, ls=':', c='k'); plt.text(10.2, ε_arr[0]*.3, "10 r_g")
plt.xlabel(r"$\hat r=r/r_g$"); plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime  (λ∝K·ℓ_P^2)")
plt.legend(); plt.grid(ls='--', alpha=.3); plt.tight_layout(); plt.show()

# Cell 8 ─ δc_g/c @ 10 r_g -----------------------------------------
f_LISA = 1e-2   # Hz
k_hat  = 2*pi*f_LISA * r_g_m / c
idx10  = np.argmin(abs(x_arr-10))
eps10  = ε_arr[idx10]
delta_c = (eps10**2 * (a_hat/10)**2)/(2*k_hat**2)

print(f"\nAt 10 r_g: ε ≈ {eps10:.2e}")
print(f"Predicted δc_g/c ≈ {delta_c:.2e}")